In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
from config import user, password

In [3]:
import pymysql
pymysql.install_as_MySQLdb()
engine = create_engine('mysql://' + user + ':' + password + '@imdb-project.cj1m13sfkxlg.us-east-2.rds.amazonaws.com:3306/gundata')


In [4]:
#Query records
data = engine.execute("SELECT Year(Date), COALESCE(state, 'Total') as st, SUM(n_killed) as Gun_Deaths FROM gundata GROUP BY Year(Date),st WITH ROLLUP")
query1 = "SELECT Year(Date), COALESCE(state, 'Total') as st, SUM(n_killed) as Gun_Deaths FROM gundata GROUP BY Year(Date),st WITH ROLLUP"

In [6]:
for record in data:
    print(record)


(2013, 'Alabama', Decimal('3'))
(2013, 'Arizona', Decimal('8'))
(2013, 'California', Decimal('46'))
(2013, 'Colorado', Decimal('4'))
(2013, 'Connecticut', Decimal('2'))
(2013, 'Delaware', Decimal('3'))
(2013, 'District of Columbia', Decimal('12'))
(2013, 'Florida', Decimal('22'))
(2013, 'Georgia', Decimal('5'))
(2013, 'Illinois', Decimal('14'))
(2013, 'Indiana', Decimal('9'))
(2013, 'Kansas', Decimal('6'))
(2013, 'Kentucky', Decimal('4'))
(2013, 'Louisiana', Decimal('8'))
(2013, 'Maryland', Decimal('5'))
(2013, 'Michigan', Decimal('9'))
(2013, 'Minnesota', Decimal('3'))
(2013, 'Mississippi', Decimal('2'))
(2013, 'Missouri', Decimal('9'))
(2013, 'Nebraska', Decimal('0'))
(2013, 'Nevada', Decimal('0'))
(2013, 'New Jersey', Decimal('12'))
(2013, 'New Mexico', Decimal('5'))
(2013, 'New York', Decimal('15'))
(2013, 'North Carolina', Decimal('5'))
(2013, 'Ohio', Decimal('10'))
(2013, 'Oklahoma', Decimal('14'))
(2013, 'Pennsylvania', Decimal('13'))
(2013, 'Rhode Island', Decimal('1'))
(2013, 

In [6]:
df1 = pd.read_sql_query(query1,engine)


In [7]:
df1.to_json(path_or_buf='overall_gun_deaths_by_year.json',orient='records')

In [11]:
query2 = """
SELECT year(gundata.date),gundata.state,city_or_county,latitude,longitude,sum(n_killed) FROM gundata
LEFT JOIN		(SELECT state,year(date)
				FROM gundata
				GROUP BY state,year(date)
				ORDER BY SUM(n_killed) DESC
                LIMIT 5) as sub
ON gundata.state = sub.state
WHERE sub.state IS NOT NULL
GROUP BY city_or_county, year(gundata.date)
ORDER BY sum(n_killed) DESC;
"""


df2 = pd.read_sql_query(query2,engine)
df2.to_json(path_or_buf='Nationwide_gun_deaths.json',orient='records')


In [22]:
#Used Stored Procedure
df3 = pd.read_sql_query("call Cause_Of_Gun_Deaths()", engine)
df3.to_json(path_or_buf='Cause_of_Gun_Deaths_by_City_State_Year.json',orient='records')


ValueError: unsupported format character 'm' (0x6d) at index 41